# Tandem Repeat analysis
#### Written by Jigar N. Bandaria

#### This is the data that I obtain after all the filters. This data was obtainted by concatenating the chr*_with_overlap.fa file into all_overlap.fa

Here the sequences that we have generated are analyzed to remove sequences that were counted twice. This occurs because in all the previous steps we had not worried about reverse complements. So here we will remove those sequences that are reverse complement.

In [35]:
import pandas as pd
import numpy as np

In [15]:
#Reading all the sequences for all the chromosomes
data = pd.read_table('chrom_only/all_overlap.fa',header=None,usecols=[0],names=['Sequence'])
data.head()

,Sequence
0,TGGACAGGCTCTCTAGA
1,ACAGGCTCTCTAGAAGG
2,CTCTCTAGAAGGTAGAC
3,TAGACAGGCTCTCTAGA
4,CTCTCTAGAAGGTAGAC


In [22]:
#These are the total number of hotspots that are present in the file.
list1 = data['Sequence'].value_counts().index.tolist()
len(list1)

73112

Below I first convert all the sequences into reverse complement and then create a dataframe that has 2 columns. One is the sequence and other is it reverse complement.

In [23]:
complement = {'A':'T','T':'A','C':'G','G':'C'} #dictionary for conversion

def rev_comp(x):#function for reverse complement
    tmp_rev = [complement[a] for a in x[::-1]]
    return ''.join(tmp_rev)

In [25]:
list2 = [rev_comp(b) for b in list1] #create the reverse complement as a new list

In [26]:
frm1 = pd.DataFrame({'fwd':list1,'rev':list2}) #convert both list of fwd and rev to pandas Dataframe
frm1.head()

,fwd,rev
0,GGACCATTCCTTCAGGA,TCCTGAAGGAATGGTCC
1,CACTCAATACACGACTC,GAGTCGTGTATTGAGTG
2,ATGGGCCCAGGTAAGCA,TGCTTACCTGGGCCCAT
3,TTCTTCAGGATGGGCCC,GGGCCCATCCTGAAGAA
4,TCTGTATGATATCACAG,CTGTGATATCATACAGA


In [ ]:
mask1 = frm1.fwd.isin(list2) # compare for the reverse complements. The number should be 2n

In [29]:
sum(mask1) # total sequences that are reverse complement of each other. Reduce this by half.

4634

In [30]:
rev_sequences = frm1[mask1] # Selecting only the ones that are reverse complement
rev_sequences[-15:]

,fwd,rev
72829,GGAATCTGATCATAAAG,CTTTATGATCAGATTCC
72852,CTCACACCCCTCAGGGC,GCCCTGAGGGGTGTGAG
72861,ACACGGTGCGCACAGCC,GGCTGTGCGCACCGTGT
72863,TGCGTGTGGGGTGTCCT,AGGACACCCCACACGCA
72888,CAGGGCCTCTAACAGCA,TGCTGTTAGAGGCCCTG
72905,GGAGCTGACCCAGCTGA,TCAGCTGGGTCAGCTCC
72945,ACCCCTTCCTAAGTGCT,AGCACTTAGGAAGGGGT
72955,TCGATGTTGGGGAACCC,GGGTTCCCCAACATCGA
72961,AAAGGAGGGATGGGTGG,CCACCCATCCCTCCTTT
73011,CCAGAGCATAATCTCCC,GGGAGATTATGCTCTGG


Since all of the sequences in rev_sequences are present 2 times, I remove one of the entry so that only one copy remains. Below is the code that retains only one copy of the sequences.

In [31]:
non_match=[]
for i in range(len(rev_sequences.fwd)):
    for j in range (i,len(rev_sequences.rev)):
        if (rev_sequences.iloc[i][0]==rev_sequences.iloc[j][1]):
            
            non_match.append(rev_sequences.index[i])
            if (i%100==0):
                print (i,j)
            
            

0 1990
100 580
500 503
800 826
900 2578
1100 1475
1200 1413
1300 2261
1400 1434
1500 1596
1600 1820
1700 2529
1800 2057
2000 2078
2100 2483
2200 3241
2300 2539
2600 3895
2800 2892
3200 4511
3400 4530
3500 4217
3600 3847
3700 4215
3800 4151
4000 4456


In [32]:
len(non_match) # this is exactly half the sequences. It contains the indices that are reverse complments
#these should be removed from  frm1.

2317

In [33]:
#save file for the reverse complement
filename = '/home/jigar/Desktop/nt_17/rev_comp_seq.txt'
fid1 = open(filename,'w')
for line in non_match:
    fid1.write(str(line))
    fid1.write('\n')
fid1.close()

In [38]:
list3 = list (np.arange(len(frm1)))

list4 = [list3.remove(x) for x in non_match]
len(list4)

2317

In [44]:
frm2 = frm1.loc[list3,:]
frm2[:5]

,fwd,rev
0,GGACCATTCCTTCAGGA,TCCTGAAGGAATGGTCC
1,CACTCAATACACGACTC,GAGTCGTGTATTGAGTG
2,ATGGGCCCAGGTAAGCA,TGCTTACCTGGGCCCAT
3,TTCTTCAGGATGGGCCC,GGGCCCATCCTGAAGAA
4,TCTGTATGATATCACAG,CTGTGATATCATACAGA


In [46]:
#saving the final list
filename1 = '/home/jigar/Desktop/nt_17/no_rev_comp.fasta'
fid2 =open(filename1,'w')
for i,line in enumerate(frm2.fwd.values.tolist()):
    fid2.write('>seq'+str(i)+'\n')
    fid2.write(line+'\n')
    
fid2.close()

In the next notebook we will align the remaining sequences back to human genome and confirm our analysis.